### 샤프지수 구하기

In [20]:
# Yahoo site로 부터 대형주 종목 데이터를 수집하여 
# 페어 트레이딩 대상 종목을 찾는다.
#
# 한국생산성본부 금융 빅데이터 전문가 과정 (금융 모델링 파트) 실습용 코드
# Written : 2018.2.5
# 제작 : 조성현
# -----------------------------------------------------------------
import pandas as pd
import numpy as np
import itertools as it
from MyUtil.YahooData import getStockDataYahoo
import os.path

df = pd.read_csv("kospi200.csv")
stocks = df.set_index('code').T.to_dict('list')

# # Yahoo site로부터 대형주 종목 데이터를 수집한다
# # 수정주가를 반영하여 삼성전자 주가 데이터를 수집한다. 가끔 안들어올 때가 있어서 10번 시도한다.
# # Yahoo site로부터 대형주 종목 데이터를 수집한다
# # 수정주가를 반영하여 삼성전자 주가 데이터를 수집한다. 가끔 안들어올 때가 있어서 10번 시도한다.
# def collectData(startDate="2007-01-01"):
#     n = 1
#     for key in stocks.keys():
#         code = str(key).rjust(6, '0')
        
#         # Yahoo site로부터 대형주 종목 데이터를 수집한다
#         s = getStockDataYahoo(code + '.KS', start=startDate)
#         print("%d) %s (%s.KS)가 수집되었습니다." % (n, stocks[key][0], code))
    
#         # 수집한 데이터를 파일에 저장한다.
#         s.to_pickle('StockData/' + code + '.KS')
#         n += 1


# 저장된 주가 데이터를 읽어와서 종목 별로 연평균 수익률, 변동성 Sharp ratio를 계산한다
def findPair():
    result = pd.DataFrame(columns=['codeA', 'codeB', 'stockA', 'stockB', 'prcCor', 'rtnCor', 'volCor', 'sum'])
    
    n = 0
    for pair in it.combinations(stocks.keys(), 2): #iteration 패키지에서 두개씩 맞춰줌 combination을 통해
        #파일 경로 지정 
        p1 = 'StockData/' + str(pair[0]).rjust(6, '0') + '.KS'
        p2 = 'StockData/' + str(pair[1]).rjust(6, '0') + '.KS'
        
        #파일 있는지 확인
        if os.path.isfile(p1) and os.path.isfile(p2):        
        # 저장된 주가 데이터를 읽어온다        
            s1 = pd.read_pickle(p1)
            s2 = pd.read_pickle(p2)
        else:
            continue

        if s1.shape[0] < 500 or s2.shape[0] < 500: #데이터 수 적으면 버리기
            continue
            
        # Pair 데이터 프레임을 구성한다 (종가와 거래량-같이 많이 사거나 팔거나 하는 경향 알기 위해)
        pdf = pd.DataFrame(columns=['stockA', 'stockB', 'volA', 'volB'])
        pdf['stockA'] = s1['Close']
        pdf['stockB'] = s2['Close']
        pdf['volA'] = s1['Volume']
        pdf['volB'] = s2['Volume']
        pdf = pdf.dropna()
        
        # 수익률 상관계수를 측정한다
        pdf['rtnA'] = np.log(pdf['stockA']) - np.log(pdf['stockA'].shift(1))
        pdf['rtnB'] = np.log(pdf['stockB']) - np.log(pdf['stockB'].shift(1))
        pdf = pdf.dropna()
        rtnCor = np.corrcoef(pdf['rtnA'], pdf['rtnB'])[1,0]

        # 로그 가격의 상관계수를 측정한다(그냥 가격으로 해도 상관없음)
        prcCor = np.corrcoef(np.log(pdf['stockA']), np.log(pdf['stockB']))[1,0]
        
        # 거래량 상관계수를 측정한다
        volCor = np.corrcoef(pdf['volA'], pdf['volB'])[1,0]
        
        # 결과를 종합한다(수익률cef + 로그가격cef + 거래량cef, 어떤 게 더 주요요인인지 확실치 않으므로) 
        sumCor = prcCor + rtnCor + volCor
        result.loc[n] = [p1, p2, stocks[pair[0]], stocks[pair[1]], prcCor, rtnCor, volCor, sumCor]
        n += 1
        
        print("%d) (%s - %s)가 처리되었습니다." % (n, pair[0], pair[1]))
        
    return result

p = findPair()
s = p.sort_values(['sum'], ascending=False).head(50)
s.to_csv('P.Pair종목(200).csv',encoding='ms949')

1) (5930 - 660)가 처리되었습니다.
2) (5930 - 68270)가 처리되었습니다.
3) (5930 - 5935)가 처리되었습니다.
4) (5930 - 5380)가 처리되었습니다.
5) (5930 - 5490)가 처리되었습니다.
6) (5930 - 207940)가 처리되었습니다.
7) (5930 - 51910)가 처리되었습니다.
8) (5930 - 35420)가 처리되었습니다.
9) (5930 - 105560)가 처리되었습니다.
10) (5930 - 28260)가 처리되었습니다.
11) (5930 - 32830)가 처리되었습니다.
12) (5930 - 12330)가 처리되었습니다.
13) (5930 - 55550)가 처리되었습니다.
14) (5930 - 34730)가 처리되었습니다.
15) (5930 - 15760)가 처리되었습니다.
16) (5930 - 18260)가 처리되었습니다.
17) (5930 - 96770)가 처리되었습니다.
18) (5930 - 17670)가 처리되었습니다.
19) (5930 - 51900)가 처리되었습니다.
20) (5930 - 66570)가 처리되었습니다.
21) (5930 - 90430)가 처리되었습니다.
22) (5930 - 3550)가 처리되었습니다.
23) (5930 - 11170)가 처리되었습니다.
24) (5930 - 86790)가 처리되었습니다.
25) (5930 - 10950)가 처리되었습니다.
26) (5930 - 33780)가 처리되었습니다.
27) (5930 - 6400)가 처리되었습니다.
28) (5930 - 270)가 처리되었습니다.
29) (5930 - 810)가 처리되었습니다.
30) (5930 - 2790)가 처리되었습니다.
31) (5930 - 30)가 처리되었습니다.
32) (5930 - 35720)가 처리되었습니다.
33) (5930 - 34220)가 처리되었습니다.
34) (5930 - 10130)가 처리되었습니다.
35) (5930 - 24110)가 처리되었습니다.
36) (59

C:\Users\user\anac\lib\site-packages\ipykernel_launcher.py:66: RuntimeWarning: invalid value encountered in log


105) (5930 - 69960)가 처리되었습니다.
106) (5930 - 5385)가 처리되었습니다.
107) (5930 - 10620)가 처리되었습니다.
108) (5930 - 6360)가 처리되었습니다.
109) (5930 - 138040)가 처리되었습니다.
110) (5930 - 3410)가 처리되었습니다.
111) (5930 - 69620)가 처리되었습니다.
112) (5930 - 139130)가 처리되었습니다.
113) (5930 - 5250)가 처리되었습니다.
114) (5930 - 4000)가 처리되었습니다.
115) (5930 - 51600)가 처리되었습니다.
116) (5930 - 150)가 처리되었습니다.
117) (5930 - 17800)가 처리되었습니다.
118) (5930 - 4370)가 처리되었습니다.
119) (5930 - 6650)가 처리되었습니다.
120) (5930 - 42670)가 처리되었습니다.
121) (5930 - 670)가 처리되었습니다.
122) (5930 - 12510)가 처리되었습니다.
123) (5930 - 240)가 처리되었습니다.
124) (5930 - 120110)가 처리되었습니다.
125) (5930 - 161890)가 처리되었습니다.
126) (5930 - 51915)가 처리되었습니다.
127) (5930 - 71840)가 처리되었습니다.
128) (5930 - 10120)가 처리되었습니다.
129) (5930 - 1800)가 처리되었습니다.
130) (5930 - 90435)가 처리되었습니다.
131) (5930 - 9420)가 처리되었습니다.
132) (5930 - 20150)가 처리되었습니다.
133) (5930 - 34020)가 처리되었습니다.
134) (5930 - 114090)가 처리되었습니다.
135) (5930 - 3520)가 처리되었습니다.
136) (5930 - 11790)가 처리되었습니다.
137) (5930 - 80)가 처리되었습니다.
138) (5930 - 12450)가 처리되

C:\Users\user\anac\lib\site-packages\ipykernel_launcher.py:66: RuntimeWarning: invalid value encountered in log


294) (660 - 10620)가 처리되었습니다.
295) (660 - 6360)가 처리되었습니다.
296) (660 - 138040)가 처리되었습니다.
297) (660 - 3410)가 처리되었습니다.
298) (660 - 69620)가 처리되었습니다.
299) (660 - 139130)가 처리되었습니다.
300) (660 - 5250)가 처리되었습니다.
301) (660 - 4000)가 처리되었습니다.
302) (660 - 51600)가 처리되었습니다.
303) (660 - 150)가 처리되었습니다.
304) (660 - 17800)가 처리되었습니다.
305) (660 - 4370)가 처리되었습니다.
306) (660 - 6650)가 처리되었습니다.
307) (660 - 42670)가 처리되었습니다.
308) (660 - 670)가 처리되었습니다.
309) (660 - 12510)가 처리되었습니다.
310) (660 - 240)가 처리되었습니다.
311) (660 - 120110)가 처리되었습니다.
312) (660 - 161890)가 처리되었습니다.
313) (660 - 51915)가 처리되었습니다.
314) (660 - 71840)가 처리되었습니다.
315) (660 - 10120)가 처리되었습니다.
316) (660 - 1800)가 처리되었습니다.
317) (660 - 90435)가 처리되었습니다.
318) (660 - 9420)가 처리되었습니다.
319) (660 - 20150)가 처리되었습니다.
320) (660 - 34020)가 처리되었습니다.
321) (660 - 114090)가 처리되었습니다.
322) (660 - 3520)가 처리되었습니다.
323) (660 - 11790)가 처리되었습니다.
324) (660 - 80)가 처리되었습니다.
325) (660 - 12450)가 처리되었습니다.
326) (660 - 11210)가 처리되었습니다.
327) (660 - 1740)가 처리되었습니다.
328) (660 - 3240)가 처리되었습니다.


...

16737) (79160 - 64350)가 처리되었습니다.
16738) (79160 - 192820)가 처리되었습니다.
16739) (79160 - 148020)가 처리되었습니다.
16740) (79160 - 115390)가 처리되었습니다.
16741) (79160 - 1630)가 처리되었습니다.
16742) (79160 - 57050)가 처리되었습니다.
16743) (79160 - 81660)가 처리되었습니다.
16744) (79160 - 39130)가 처리되었습니다.
16745) (79160 - 5300)가 처리되었습니다.
16746) (79160 - 111770)가 처리되었습니다.
16747) (79160 - 214320)가 처리되었습니다.
16748) (79160 - 180640)가 처리되었습니다.
16749) (79160 - 14820)가 처리되었습니다.
16750) (79160 - 1440)가 처리되었습니다.
16751) (79160 - 102780)가 처리되었습니다.
16752) (79160 - 2350)가 처리되었습니다.
16753) (79160 - 82640)가 처리되었습니다.
16754) (79160 - 103140)가 처리되었습니다.
16755) (79160 - 89590)가 처리되었습니다.
16756) (79160 - 5610)가 처리되었습니다.
16757) (79160 - 153130)가 처리되었습니다.
16758) (79160 - 23590)가 처리되었습니다.
16759) (79160 - 3000)가 처리되었습니다.
16760) (79160 - 229200)가 처리되었습니다.
16761) (79160 - 3090)가 처리되었습니다.
16762) (79160 - 6840)가 처리되었습니다.
16763) (79160 - 69260)가 처리되었습니다.
16764) (79160 - 6040)가 처리되었습니다.
16765) (79160 - 1120)가 처리되었습니다.
16766) (79160 - 152100)가 처리되었습니다.
16767) (7

16990) (64350 - 1630)가 처리되었습니다.
16991) (64350 - 57050)가 처리되었습니다.
16992) (64350 - 81660)가 처리되었습니다.
16993) (64350 - 39130)가 처리되었습니다.
16994) (64350 - 5300)가 처리되었습니다.
16995) (64350 - 111770)가 처리되었습니다.
16996) (64350 - 214320)가 처리되었습니다.
16997) (64350 - 180640)가 처리되었습니다.
16998) (64350 - 14820)가 처리되었습니다.
16999) (64350 - 1440)가 처리되었습니다.
17000) (64350 - 102780)가 처리되었습니다.
17001) (64350 - 2350)가 처리되었습니다.
17002) (64350 - 82640)가 처리되었습니다.
17003) (64350 - 103140)가 처리되었습니다.
17004) (64350 - 89590)가 처리되었습니다.
17005) (64350 - 5610)가 처리되었습니다.
17006) (64350 - 153130)가 처리되었습니다.
17007) (64350 - 23590)가 처리되었습니다.
17008) (64350 - 3000)가 처리되었습니다.
17009) (64350 - 229200)가 처리되었습니다.
17010) (64350 - 3090)가 처리되었습니다.
17011) (64350 - 6840)가 처리되었습니다.
17012) (64350 - 69260)가 처리되었습니다.
17013) (64350 - 6040)가 처리되었습니다.
17014) (64350 - 1120)가 처리되었습니다.
17015) (64350 - 152100)가 처리되었습니다.
17016) (64350 - 79550)가 처리되었습니다.
17017) (64350 - 20560)가 처리되었습니다.
17018) (64350 - 175330)가 처리되었습니다.
17019) (64350 - 370)가 처리되었습니다.
17020) (64350

17239) (39130 - 5300)가 처리되었습니다.
17240) (39130 - 111770)가 처리되었습니다.
17241) (39130 - 214320)가 처리되었습니다.
17242) (39130 - 180640)가 처리되었습니다.
17243) (39130 - 14820)가 처리되었습니다.
17244) (39130 - 1440)가 처리되었습니다.
17245) (39130 - 102780)가 처리되었습니다.
17246) (39130 - 2350)가 처리되었습니다.
17247) (39130 - 82640)가 처리되었습니다.
17248) (39130 - 103140)가 처리되었습니다.
17249) (39130 - 89590)가 처리되었습니다.
17250) (39130 - 5610)가 처리되었습니다.
17251) (39130 - 153130)가 처리되었습니다.
17252) (39130 - 23590)가 처리되었습니다.
17253) (39130 - 3000)가 처리되었습니다.
17254) (39130 - 229200)가 처리되었습니다.
17255) (39130 - 3090)가 처리되었습니다.
17256) (39130 - 6840)가 처리되었습니다.
17257) (39130 - 69260)가 처리되었습니다.
17258) (39130 - 6040)가 처리되었습니다.
17259) (39130 - 1120)가 처리되었습니다.
17260) (39130 - 152100)가 처리되었습니다.
17261) (39130 - 79550)가 처리되었습니다.
17262) (39130 - 20560)가 처리되었습니다.
17263) (39130 - 175330)가 처리되었습니다.
17264) (39130 - 370)가 처리되었습니다.
17265) (39130 - 71320)가 처리되었습니다.
17266) (39130 - 10780)가 처리되었습니다.
17267) (39130 - 1230)가 처리되었습니다.
17268) (39130 - 52690)가 처리되었습니다.
17269) (39130

17488) (2350 - 52690)가 처리되었습니다.
17489) (2350 - 1430)가 처리되었습니다.
17490) (2350 - 3300)가 처리되었습니다.
17491) (82640 - 103140)가 처리되었습니다.
17492) (82640 - 89590)가 처리되었습니다.
17493) (82640 - 5610)가 처리되었습니다.
17494) (82640 - 153130)가 처리되었습니다.
17495) (82640 - 23590)가 처리되었습니다.
17496) (82640 - 3000)가 처리되었습니다.
17497) (82640 - 229200)가 처리되었습니다.
17498) (82640 - 3090)가 처리되었습니다.
17499) (82640 - 6840)가 처리되었습니다.
17500) (82640 - 69260)가 처리되었습니다.
17501) (82640 - 6040)가 처리되었습니다.
17502) (82640 - 1120)가 처리되었습니다.
17503) (82640 - 152100)가 처리되었습니다.
17504) (82640 - 79550)가 처리되었습니다.
17505) (82640 - 20560)가 처리되었습니다.
17506) (82640 - 175330)가 처리되었습니다.
17507) (82640 - 370)가 처리되었습니다.
17508) (82640 - 71320)가 처리되었습니다.
17509) (82640 - 10780)가 처리되었습니다.
17510) (82640 - 1230)가 처리되었습니다.
17511) (82640 - 52690)가 처리되었습니다.
17512) (82640 - 1430)가 처리되었습니다.
17513) (82640 - 3300)가 처리되었습니다.
17514) (103140 - 89590)가 처리되었습니다.
17515) (103140 - 5610)가 처리되었습니다.
17516) (103140 - 153130)가 처리되었습니다.
17517) (103140 - 23590)가 처리되었습니다.
17518) (103140 - 

17743) (175330 - 52690)가 처리되었습니다.
17744) (175330 - 1430)가 처리되었습니다.
17745) (175330 - 3300)가 처리되었습니다.
17746) (370 - 71320)가 처리되었습니다.
17747) (370 - 10780)가 처리되었습니다.
17748) (370 - 1230)가 처리되었습니다.
17749) (370 - 52690)가 처리되었습니다.
17750) (370 - 1430)가 처리되었습니다.
17751) (370 - 3300)가 처리되었습니다.
17752) (71320 - 10780)가 처리되었습니다.
17753) (71320 - 1230)가 처리되었습니다.
17754) (71320 - 52690)가 처리되었습니다.
17755) (71320 - 1430)가 처리되었습니다.
17756) (71320 - 3300)가 처리되었습니다.
17757) (10780 - 1230)가 처리되었습니다.
17758) (10780 - 52690)가 처리되었습니다.
17759) (10780 - 1430)가 처리되었습니다.
17760) (10780 - 3300)가 처리되었습니다.
17761) (1230 - 52690)가 처리되었습니다.
17762) (1230 - 1430)가 처리되었습니다.
17763) (1230 - 3300)가 처리되었습니다.
17764) (52690 - 1430)가 처리되었습니다.
17765) (52690 - 3300)가 처리되었습니다.
17766) (1430 - 3300)가 처리되었습니다.


In [21]:
p = findPair()
s = p.sort_values(['sum'], ascending=False).head(50)
s.to_csv('P.Pair종목(200).csv',encoding='ms949')

1) (5930 - 660)가 처리되었습니다.
2) (5930 - 68270)가 처리되었습니다.
3) (5930 - 5935)가 처리되었습니다.
4) (5930 - 5380)가 처리되었습니다.
5) (5930 - 5490)가 처리되었습니다.
6) (5930 - 207940)가 처리되었습니다.
7) (5930 - 51910)가 처리되었습니다.
8) (5930 - 35420)가 처리되었습니다.
9) (5930 - 105560)가 처리되었습니다.
10) (5930 - 28260)가 처리되었습니다.
11) (5930 - 32830)가 처리되었습니다.
12) (5930 - 12330)가 처리되었습니다.
13) (5930 - 55550)가 처리되었습니다.
14) (5930 - 34730)가 처리되었습니다.
15) (5930 - 15760)가 처리되었습니다.
16) (5930 - 18260)가 처리되었습니다.
17) (5930 - 96770)가 처리되었습니다.
18) (5930 - 17670)가 처리되었습니다.
19) (5930 - 51900)가 처리되었습니다.
20) (5930 - 66570)가 처리되었습니다.
21) (5930 - 90430)가 처리되었습니다.
22) (5930 - 3550)가 처리되었습니다.
23) (5930 - 11170)가 처리되었습니다.
24) (5930 - 86790)가 처리되었습니다.
25) (5930 - 10950)가 처리되었습니다.
26) (5930 - 33780)가 처리되었습니다.
27) (5930 - 6400)가 처리되었습니다.
28) (5930 - 270)가 처리되었습니다.
29) (5930 - 810)가 처리되었습니다.
30) (5930 - 2790)가 처리되었습니다.
31) (5930 - 30)가 처리되었습니다.
32) (5930 - 35720)가 처리되었습니다.
33) (5930 - 34220)가 처리되었습니다.
34) (5930 - 10130)가 처리되었습니다.
35) (5930 - 24110)가 처리되었습니다.
36) (59

C:\Users\user\anac\lib\site-packages\ipykernel_launcher.py:66: RuntimeWarning: invalid value encountered in log


103) (5930 - 30000)가 처리되었습니다.
104) (5930 - 60980)가 처리되었습니다.
105) (5930 - 69960)가 처리되었습니다.
106) (5930 - 5385)가 처리되었습니다.
107) (5930 - 10620)가 처리되었습니다.
108) (5930 - 6360)가 처리되었습니다.
109) (5930 - 138040)가 처리되었습니다.
110) (5930 - 3410)가 처리되었습니다.
111) (5930 - 69620)가 처리되었습니다.
112) (5930 - 139130)가 처리되었습니다.
113) (5930 - 5250)가 처리되었습니다.
114) (5930 - 4000)가 처리되었습니다.
115) (5930 - 51600)가 처리되었습니다.
116) (5930 - 150)가 처리되었습니다.
117) (5930 - 17800)가 처리되었습니다.
118) (5930 - 4370)가 처리되었습니다.
119) (5930 - 6650)가 처리되었습니다.
120) (5930 - 42670)가 처리되었습니다.
121) (5930 - 670)가 처리되었습니다.
122) (5930 - 12510)가 처리되었습니다.
123) (5930 - 240)가 처리되었습니다.
124) (5930 - 120110)가 처리되었습니다.
125) (5930 - 161890)가 처리되었습니다.
126) (5930 - 51915)가 처리되었습니다.
127) (5930 - 71840)가 처리되었습니다.
128) (5930 - 10120)가 처리되었습니다.
129) (5930 - 1800)가 처리되었습니다.
130) (5930 - 90435)가 처리되었습니다.
131) (5930 - 9420)가 처리되었습니다.
132) (5930 - 20150)가 처리되었습니다.
133) (5930 - 34020)가 처리되었습니다.
134) (5930 - 114090)가 처리되었습니다.
135) (5930 - 3520)가 처리되었습니다.
136) (5930 - 11790)가 

C:\Users\user\anac\lib\site-packages\ipykernel_launcher.py:66: RuntimeWarning: invalid value encountered in log



294) (660 - 10620)가 처리되었습니다.
295) (660 - 6360)가 처리되었습니다.
296) (660 - 138040)가 처리되었습니다.
297) (660 - 3410)가 처리되었습니다.
298) (660 - 69620)가 처리되었습니다.
299) (660 - 139130)가 처리되었습니다.
300) (660 - 5250)가 처리되었습니다.
301) (660 - 4000)가 처리되었습니다.
302) (660 - 51600)가 처리되었습니다.
303) (660 - 150)가 처리되었습니다.
304) (660 - 17800)가 처리되었습니다.
305) (660 - 4370)가 처리되었습니다.
306) (660 - 6650)가 처리되었습니다.
307) (660 - 42670)가 처리되었습니다.
308) (660 - 670)가 처리되었습니다.
309) (660 - 12510)가 처리되었습니다.
310) (660 - 240)가 처리되었습니다.
311) (660 - 120110)가 처리되었습니다.
312) (660 - 161890)가 처리되었습니다.
313) (660 - 51915)가 처리되었습니다.
314) (660 - 71840)가 처리되었습니다.
315) (660 - 10120)가 처리되었습니다.
316) (660 - 1800)가 처리되었습니다.
317) (660 - 90435)가 처리되었습니다.
318) (660 - 9420)가 처리되었습니다.
319) (660 - 20150)가 처리되었습니다.
320) (660 - 34020)가 처리되었습니다.
321) (660 - 114090)가 처리되었습니다.
322) (660 - 3520)가 처리되었습니다.
323) (660 - 11790)가 처리되었습니다.
324) (660 - 80)가 처리되었습니다.
325) (660 - 12450)가 처리되었습니다.
326) (660 - 11210)가 처리되었습니다.
327) (660 - 1740)가 처리되었습니다.
328) (660 - 3240)가 처리되었습니다.

...

16748) (79160 - 180640)가 처리되었습니다.
16749) (79160 - 14820)가 처리되었습니다.
16750) (79160 - 1440)가 처리되었습니다.
16751) (79160 - 102780)가 처리되었습니다.
16752) (79160 - 2350)가 처리되었습니다.
16753) (79160 - 82640)가 처리되었습니다.
16754) (79160 - 103140)가 처리되었습니다.
16755) (79160 - 89590)가 처리되었습니다.
16756) (79160 - 5610)가 처리되었습니다.
16757) (79160 - 153130)가 처리되었습니다.
16758) (79160 - 23590)가 처리되었습니다.
16759) (79160 - 3000)가 처리되었습니다.
16760) (79160 - 229200)가 처리되었습니다.
16761) (79160 - 3090)가 처리되었습니다.
16762) (79160 - 6840)가 처리되었습니다.
16763) (79160 - 69260)가 처리되었습니다.
16764) (79160 - 6040)가 처리되었습니다.
16765) (79160 - 1120)가 처리되었습니다.
16766) (79160 - 152100)가 처리되었습니다.
16767) (79160 - 79550)가 처리되었습니다.
16768) (79160 - 20560)가 처리되었습니다.
16769) (79160 - 175330)가 처리되었습니다.
16770) (79160 - 370)가 처리되었습니다.
16771) (79160 - 71320)가 처리되었습니다.
16772) (79160 - 10780)가 처리되었습니다.
16773) (79160 - 1230)가 처리되었습니다.
16774) (79160 - 52690)가 처리되었습니다.
16775) (79160 - 1430)가 처리되었습니다.
16776) (79160 - 3300)가 처리되었습니다.
16777) (3690 - 181710)가 처리되었습니다.
16778) (3690 - 5

17002) (64350 - 82640)가 처리되었습니다.
17003) (64350 - 103140)가 처리되었습니다.
17004) (64350 - 89590)가 처리되었습니다.
17005) (64350 - 5610)가 처리되었습니다.
17006) (64350 - 153130)가 처리되었습니다.
17007) (64350 - 23590)가 처리되었습니다.
17008) (64350 - 3000)가 처리되었습니다.
17009) (64350 - 229200)가 처리되었습니다.
17010) (64350 - 3090)가 처리되었습니다.
17011) (64350 - 6840)가 처리되었습니다.
17012) (64350 - 69260)가 처리되었습니다.
17013) (64350 - 6040)가 처리되었습니다.
17014) (64350 - 1120)가 처리되었습니다.
17015) (64350 - 152100)가 처리되었습니다.
17016) (64350 - 79550)가 처리되었습니다.
17017) (64350 - 20560)가 처리되었습니다.
17018) (64350 - 175330)가 처리되었습니다.
17019) (64350 - 370)가 처리되었습니다.
17020) (64350 - 71320)가 처리되었습니다.
17021) (64350 - 10780)가 처리되었습니다.
17022) (64350 - 1230)가 처리되었습니다.
17023) (64350 - 52690)가 처리되었습니다.
17024) (64350 - 1430)가 처리되었습니다.
17025) (64350 - 3300)가 처리되었습니다.
17026) (192820 - 148020)가 처리되었습니다.
17027) (192820 - 115390)가 처리되었습니다.
17028) (192820 - 1630)가 처리되었습니다.
17029) (192820 - 57050)가 처리되었습니다.
17030) (192820 - 81660)가 처리되었습니다.
17031) (192820 - 39130)가 처리되었습니다.
17032) (1

17249) (39130 - 89590)가 처리되었습니다.
17250) (39130 - 5610)가 처리되었습니다.
17251) (39130 - 153130)가 처리되었습니다.
17252) (39130 - 23590)가 처리되었습니다.
17253) (39130 - 3000)가 처리되었습니다.
17254) (39130 - 229200)가 처리되었습니다.
17255) (39130 - 3090)가 처리되었습니다.
17256) (39130 - 6840)가 처리되었습니다.
17257) (39130 - 69260)가 처리되었습니다.
17258) (39130 - 6040)가 처리되었습니다.
17259) (39130 - 1120)가 처리되었습니다.
17260) (39130 - 152100)가 처리되었습니다.
17261) (39130 - 79550)가 처리되었습니다.
17262) (39130 - 20560)가 처리되었습니다.
17263) (39130 - 175330)가 처리되었습니다.
17264) (39130 - 370)가 처리되었습니다.
17265) (39130 - 71320)가 처리되었습니다.
17266) (39130 - 10780)가 처리되었습니다.
17267) (39130 - 1230)가 처리되었습니다.
17268) (39130 - 52690)가 처리되었습니다.
17269) (39130 - 1430)가 처리되었습니다.
17270) (39130 - 3300)가 처리되었습니다.
17271) (5300 - 111770)가 처리되었습니다.
17272) (5300 - 214320)가 처리되었습니다.
17273) (5300 - 180640)가 처리되었습니다.
17274) (5300 - 14820)가 처리되었습니다.
17275) (5300 - 1440)가 처리되었습니다.
17276) (5300 - 102780)가 처리되었습니다.
17277) (5300 - 2350)가 처리되었습니다.
17278) (5300 - 82640)가 처리되었습니다.
17279) (5300 - 103140)가

17498) (82640 - 3090)가 처리되었습니다.
17499) (82640 - 6840)가 처리되었습니다.
17500) (82640 - 69260)가 처리되었습니다.
17501) (82640 - 6040)가 처리되었습니다.
17502) (82640 - 1120)가 처리되었습니다.
17503) (82640 - 152100)가 처리되었습니다.
17504) (82640 - 79550)가 처리되었습니다.
17505) (82640 - 20560)가 처리되었습니다.
17506) (82640 - 175330)가 처리되었습니다.
17507) (82640 - 370)가 처리되었습니다.
17508) (82640 - 71320)가 처리되었습니다.
17509) (82640 - 10780)가 처리되었습니다.
17510) (82640 - 1230)가 처리되었습니다.
17511) (82640 - 52690)가 처리되었습니다.
17512) (82640 - 1430)가 처리되었습니다.
17513) (82640 - 3300)가 처리되었습니다.
17514) (103140 - 89590)가 처리되었습니다.
17515) (103140 - 5610)가 처리되었습니다.
17516) (103140 - 153130)가 처리되었습니다.
17517) (103140 - 23590)가 처리되었습니다.
17518) (103140 - 3000)가 처리되었습니다.
17519) (103140 - 229200)가 처리되었습니다.
17520) (103140 - 3090)가 처리되었습니다.
17521) (103140 - 6840)가 처리되었습니다.
17522) (103140 - 69260)가 처리되었습니다.
17523) (103140 - 6040)가 처리되었습니다.
17524) (103140 - 1120)가 처리되었습니다.
17525) (103140 - 152100)가 처리되었습니다.
17526) (103140 - 79550)가 처리되었습니다.
17527) (103140 - 20560)가 처리되었습니다.
17528)

17749) (370 - 52690)가 처리되었습니다.
17750) (370 - 1430)가 처리되었습니다.
17751) (370 - 3300)가 처리되었습니다.
17752) (71320 - 10780)가 처리되었습니다.
17753) (71320 - 1230)가 처리되었습니다.
17754) (71320 - 52690)가 처리되었습니다.
17755) (71320 - 1430)가 처리되었습니다.
17756) (71320 - 3300)가 처리되었습니다.
17757) (10780 - 1230)가 처리되었습니다.
17758) (10780 - 52690)가 처리되었습니다.
17759) (10780 - 1430)가 처리되었습니다.
17760) (10780 - 3300)가 처리되었습니다.
17761) (1230 - 52690)가 처리되었습니다.
17762) (1230 - 1430)가 처리되었습니다.
17763) (1230 - 3300)가 처리되었습니다.
17764) (52690 - 1430)가 처리되었습니다.
17765) (52690 - 3300)가 처리되었습니다.
17766) (1430 - 3300)가 처리되었습니다.
